In [18]:
# Import paakage
using Distributions, Random, DelimitedFiles, Shell, LinearAlgebra, Glob, ZipFile, Interpolations

In [ ]:
# randon3dOrientationTensorGenerator
# Randomly sample a 3D random reference orientation tensor.
# Randomly sample a reference fiber volume fraction from a given value +0-50%.
#
# fibervf:          Base volume fraction of fiber in the composite material.
# path:             File path including file name and number (will append .txt).
# path2:            File path for RVE size, including file name and number (will append .txt).
#
function randon3dOrientationTensorGenerator(fibervf,path,path2="")
    # Random sampling of principal components.
    x = diff(sort([0;rand(2);1]))
    L = diagm(x)
    # Random sampling of rotation parameters.
    theta = 2*pi*rand()
    phi = 2*pi*rand()
    z = rand()

    # Generate rotation matrix around z-axis.
    R = [cos(theta) sin(theta) 0;
    -sin(theta) cos(theta) 0;
    0 0 1]

    # Generate mirrored householder matrix.
    v = [cos(phi)*sqrt(z); sin(phi)*sqrt(z); sqrt(1-z)]
    P = 2*v*v' - Matrix(I,3,3)

    # Total rotation matrix which is sampled uniformly from SO(3)
    M = P*R
    # Uniformly random sampled orientation tensor.
    a = M*L*M'

    sizeRVE = [4.5e-4 4.5e-4 4.5e-4]

    # Randomly sample a volume fraction between initial volume fraction + 0-50%
    vf = string(parse(Float64,fibervf) + parse(Float64,fibervf)*0.5*rand())
    # Write to file!
    header = ["a11" "a22" "a33" "a12" "a13" "a23" "vf"]
    currentPath = path*".txt"
    open(currentPath, "w") do io
        writedlm(io,vcat(header,hcat(a[1,1],a[2,2],a[3,3],a[1,2],a[1,3],a[2,3],
        vf)))
    end
    header = ["rve_size_x" "rve_size_y" "rve_size_z"]
    currentPath = path2*".txt"
    open(currentPath, "w") do io
        writedlm(io,vcat(header,hcat(sizeRVE[1],sizeRVE[2],sizeRVE[3])))
    end
    return nothing
end

In [ ]:
# strainPathGenerator
# Generates a path in 6D strain space and prints to a tab delimited file.
# Picks n_drift directions and walks n_step steps in those directions with
# added noise. The noise vector is multiplied by the variable noise.
# The directions and noise is sampled from a normal distribution with mean
# 0 and standard deviation 1. The output data file is formatted column wise
# given in the order "time" "e11" "e22" "e33" "2*e12" "2*e23" "2*e13".
#
# noise:    Noise multiplicative factor
# n_step:   Number of random steps per directions
# n_drift:  Number of drift directions
# EPS:      Maximum admissible strain component
# t:        Time variable
# path:     File path including file name and number (will append .txt).
# uni:      If true; sets all but one random strain component to 0.

function strainPathGenerator(noise,n_step,n_drift,EPS,t,path,uni)
    X = repeat(rand(Normal(0,1),(n_drift,6)),inner=[n_step, 1])
    X = X./sqrt.(sum((X).^2,dims=2))

    Y = rand(Normal(0,1),(n_drift*n_step,6))
    Y = noise.*Y./sqrt.(sum((Y).^2,dims=2))

    Eps = vcat(zeros(1,6),cumsum(X+Y,dims=1))

    # For unidirectional strain, delete all components except for one.
    if uni
        zeroIndex = [1;2;3;4;5;6]
        deleteat!(zeroIndex,rand(1:6))
        Eps[:,zeroIndex] .= 0
    end

    s = maximum(abs.(Eps))
    eps = Eps.*EPS./s
    # Input format is shearstrain*2, therefore those components are scaled.
    eps[:,4:end] = eps[:,4:end]*2

    header = ["time" "e11" "e22" "e33" "2*e12" "2*e23" "2*e13"]
    currentPath =path*".txt"
    open(currentPath, "w") do io
        writedlm(io, vcat(header,hcat(t,eps)))
    end

    return nothing
end

In [ ]:
# writedaf
# Writes *.mat for analysis of linear exponential isotropic hardening matrix
# with elastic fibers. The loading conditions are loaded from a strain path
# file. The loading conditions are general 3D loading, prescribed 6D strain.
#
# inPath:                      The path of the input strain file.
# inPath2:                     The path of the input orientation file.
# inPath3:                     The path of the RVE size file.
# outPath:                     The path of the output *.daf file.
# finalTime:                   Final time of the analysis.
# maxTimeInc:                  Maximum integration time step.
# minTimeInc:                  Minimum integration time step.
# initialTimeInc:              Initial time inctement.
# youngFiber:                  Young's modulus of fiber.
# poissonFiber:                Poisson's ratio of fiber.
# fiberDiameter:          	   Fiber diameter.
# fiberSize:				   Fiber size/ length.
# fiberOrientation:            Fiber orientation.
# youngMatrix:                 Young's modulus of matrix.
# poissonMatrix:               Poisson's ratio of matrix.
# yieldMatrix:                 Yield stress of matrix.
# hardeningModulusMatrix:      Hardening modulus R_inf of matrix.
# hardeningExponentMatrix:     Hardening exponent m of matrix.
# hardeningModulus2Matrix:     Linear hardening modulus of matrix.
#
function writedaf(inPath,inPath2,inPath3,outPath,finalTime,maxTimeInc,minTimeInc,initialTimeInc,
    youngFiber,poissonFiber,fiberDiameter,fiberSize,fiberOrientation,
    youngMatrix,poissonMatrix,yieldMatrix,hardeningModulusMatrix,
    hardeningExponentMatrix,hardeningModulus2Matrix)

    analysisName = outPath[findlast("/",outPath)[1]+1:end-4]

    separator = "##########################################"

    # Begin by reading strain data file.
    strainData = open(inPath, "r") do io
        readdlm(io, '\t')
    end

    # Read orientation tensor if applicable.
    orientationTensor = open(inPath2, "r") do io
        readdlm(io, '\t')
    end
    fiberVolumefraction = string.(orientationTensor[end])
    matrixVolumefraction = string.(1-orientationTensor[end])
    a = string.(orientationTensor[2,:])
    
    sizeRVE = readdlm(inPath3,skipstart=1)
    sizeRVEx = string(sizeRVE[1])
    sizeRVEy = string(sizeRVE[2])
    sizeRVEz = string(sizeRVE[3])

    # Define entries for sections of *.mat
    fiberMATERIAL = [""; separator; "MATERIAL"; "name = Fiber"; "type = elastic";
    "elastic_model = isotropic"; "Young = "*youngFiber; "Poisson = "*poissonFiber; ""]

    matrixMATERIAL = [separator; "MATERIAL"; "name = Matrix";
    "type = J2_plasticity"; "consistent_tangent = on"; "elastic_model = isotropic";
    "Young = "*youngMatrix; "Poisson = "*poissonMatrix;
    "yield_stress = "*yieldMatrix; "hardening_model = exponential_linear";
    "hardening_modulus = "*hardeningModulusMatrix;
    "hardening_exponent = "*hardeningExponentMatrix;
    "hardening_modulus2 = "*hardeningModulus2Matrix; "isotropic_method = spectral"; ""]

    if fiberOrientation == "random_3D"
        fiberPHASE = [separator; "PHASE";"name = InclusionPhase";
        "type = inclusion_fe"; "volume_fraction = "*fiberVolumefraction;
        "material = Fiber"; "inclusion_shape = cylinder";
        "phase_definition = by_size_and_diameter";
        "inclusion_diameter = "*fiberDiameter; "inclusion_size = "*fiberSize;
        "size_distribution = fixed";
        "orientation = "*fiberOrientation; "coated = no"; 
        "interface_behavior = perfectly_bonded"; "clustering = no"; 
        "allow_size_reduction = no"; "track_percolation_onset = no"; 
        "stop_at_percolation = no"; "check_final_percolation = no"; 
        "no_tie_on_fiber_tips = no"; ""]
    else
        fiberPHASE = [separator; "PHASE";"name = InclusionPhase";
        "type = inclusion_fe"; "volume_fraction = "*fiberVolumefraction;
        "material = Fiber"; "inclusion_shape = cylinder";
        "phase_definition = by_size_and_diameter";
        "inclusion_diameter = "*fiberDiameter; "inclusion_size = "*fiberSize;
        "size_distribution = fixed";
        "orientation = "*fiberOrientation; 
        "orientation_11 = "*a[1]; "orientation_22 = "*a[2];
        "orientation_33 = "*a[3]; "orientation_12 = "*a[4];
        "orientation_13 = "*a[5]; "orientation_23 = "*a[6];
        "coated = no"; "interface_behavior = perfectly_bonded"; 
        "clustering = no"; "allow_size_reduction = no"; 
        "track_percolation_onset = no"; "stop_at_percolation = no"; 
        "check_final_percolation = no"; "no_tie_on_fiber_tips = no"; ""]
    end

    matrixPHASE = [separator; "PHASE";"name = MatrixPhase"; "type = matrix";
    "volume_fraction = "*matrixVolumefraction; "material = Matrix"; ""]


    MICROSTRUCTURE = [separator; "MICROSTRUCTURE"; "name = TheMicrostructure";
    "phase = MatrixPhase"; "phase = InclusionPhase"; ""]

    LOADING = [separator; "LOADING"; "name = Mechanical"; "type = strain";
    "load = General_3D"; "initial_strain_11 = 0.0e+00"; "strain_11 = 1.0e+00";
    "initial_strain_22 = 0.0e+00"; "strain_22 = 1.0e+00";
    "initial_strain_33 = 0.0e+00"; "strain_33 = 1.0e+00";
    "initial_strain_12 = 0.0e+00"; "strain_12 = 1.0e+00";
    "initial_strain_23 = 0.0e+00"; "strain_23 = 1.0e+00";
    "initial_strain_13 = 0.0e+00"; "strain_13 = 1.0e+00";
    "history = user_defined"; "history_component_11 = e11";
    "history_component_11_value = relative";
    "history_component_22 = e22"; "history_component_22_value = relative";
    "history_component_33 = e33"; "history_component_33_value = relative";
    "history_component_12 = e12"; "history_component_12_value = relative";
    "history_component_23 = e23"; "history_component_23_value = relative";
    "history_component_13 = e13"; "history_component_13_value = relative";
    "quasi_static = on"; ""]

    RVE = [separator; "RVE"; "type = classical"; "microstructure = TheMicrostructure"; ""]

    MESH = [separator; "MESH"; "mesh_type = non_conforming"; "automatic_mesh_sizing = on"; ""]

    ANALYSISFE = [separator; "ANALYSISFE"; "name = "*analysisName; "type = mechanical";
    "loading_name = Mechanical";
    "final_time = "*finalTime; "max_time_inc = "*maxTimeInc;
    "min_time_inc = "*minTimeInc; "finite_strain = off";
    "initial_time_inc = "*initialTimeInc; "max_number_increment = 3000";
    "rve_size_definition = user_defined"; "rve_dimension = 3d";
    "size_rve_x = "*sizeRVEx; "size_rve_y = "*sizeRVEy; "size_rve_z = "*sizeRVEz;
    "periodic = yes"; "generation_sequence = proportional"; "generate_matrix = no";
    "track_global_percolation_onset = no"; "stop_at_global_percolation = no";
    "check_final_global_percolation = no"; "random_seed_type = automatic";
    "random_seed = -121641029"; "fe_solver = DigimatSpectralSolver"; 
    "default_timestepping = no"; "precision = single"; "processor = gpu"; "linear_solver_tol = 1.0e-08";
    "nonlinear_solver_norm2_tol = 1.0e-04"; "nonlinear_solver_min_iter = 2"; 
    "nonlinear_solver_max_iter = 10"; "void_to_matrix_stiffness_ratio = 1.0e-04";
    "green_operator = default"; ""]

    GLOBAL_SETTINGS = [separator; "GLOBAL_SETTINGS"; "allow_interpenetration = no"; 
    "allow_coating_interpenetration = no"; "allow_rim_interpenetration = no"; 
    "use_median_plane_interpenetration = no"; "cubic_architecture = no"; 
    "apply_perturbation = no"; "favor_orientation_over_fraction = no";
    "minimum_relative_distance_wrt_diameter = 5.0e-02"; "minimum_relative_vol = 5.0e-02"; 
    "max_number_of_tests = 200000"; "OT_norm_tol = 1.0e-01"; "max_number_of_geometry_attempts = 10";
    "minimum_rel_dist_incl_to_face = 0.0e+00"; "maximum_interpenetration_amount = 1.0e+00";
    "random_fiber_perturbation_no_transverse_perturbation = no"; "default_geometric_options = no";
    "remove_unconnected_matrix_regions = no"; ""]

    OUTPUT = [separator; "OUTPUT"; "stress_output = on"; 
    "strain_output = on"; "sdv_output = off"; 
    "stiffness_output = off"; "nb_histogram_class = 50"; 
    "macro_results_output = on"; ""]

    point = repeat(["point = "],outer=length(strainData[2:end,1]))

    e11FUNCTION = [separator; "FUNCTION"; "name = e11"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,2]); ""]

    e22FUNCTION = [separator; "FUNCTION"; "name = e22"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,3]); ""]

    e33FUNCTION = [separator; "FUNCTION"; "name = e33"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,4]); ""]

    e12FUNCTION = [separator; "FUNCTION"; "name = e12"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,5]); ""]

    e23FUNCTION = [separator; "FUNCTION"; "name = e23"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,6]); ""]

    e13FUNCTION = [separator; "FUNCTION"; "name = e13"; "type = piecewise_linear";
    point.*string.(strainData[2:end,1]).*",".*string.(strainData[2:end,7])]

    open(outPath, "w") do io
        writedlm(io,fiberMATERIAL)
        writedlm(io,matrixMATERIAL)
        writedlm(io,matrixPHASE)
        writedlm(io,fiberPHASE)
        writedlm(io,MICROSTRUCTURE)
        writedlm(io,LOADING)
        writedlm(io,RVE)
        writedlm(io,MESH)
        writedlm(io,ANALYSISFE)
        writedlm(io,GLOBAL_SETTINGS)
        writedlm(io,OUTPUT)
        writedlm(io,e11FUNCTION)
        writedlm(io,e22FUNCTION)
        writedlm(io,e33FUNCTION)
        writedlm(io,e12FUNCTION)
        writedlm(io,e23FUNCTION)
        writedlm(io,e13FUNCTION)
    end
end

In [ ]:
# RunDigimatFEwithTimeLimit
# Run Digimat-FE FFT simulation with time limits.
#
# i:                Simulation number.
# workingPath:      Path to the .daf file.
# maxTime:          Maximum allowable time in second.
# checkTime:        Time (in second) after submission of the simulation to check has it begins. 
#
function RunDigimatFEwithTimeLimit(i, workingPath, maxTime, checkTime)
    command1 = "C:/MSC.Software/Digimat/2022.4/DigimatFE/exec/DigimatFE.bat" 
    command2 = "-runFEWorkflow"
    command3 = "input="*workingPath*"analysis_"*string(i)*".daf"
    command4 = "workingDir="*workingPath
    
    startTime = time_ns()
    Process = run(`$command1 $command2 $command3 $command4`,wait=false)
    print("Submitted analysis_"*string(i))

    macFile = workingPath*"analysis_"*string(i)*".sts"
    checkFlag = 0

    while process_running(Process)
        sleep(60)
        timeElapsed = (time_ns()-startTime)/1e9
    
        if timeElapsed>maxTime
            print("Killing analysis_"*string(i)*" due to exceeding runtime.")
            while process_running(Process)
                kill(Process)
            end
            print("Killed analysis_"*string(i)*" due to exceeding runtime.")
        end
    
        if timeElapsed>checkTime && checkFlag==0 && !isfile(macFile)
            checkFlag = 1
            print("Killing analysis_"*string(i)*" due to check error.")
            while process_running(Process)
                kill(Process)
            end
            print("Killed analysis_"*string(i)*" due to check error.")
        end

        if timeElapsed>checkTime && checkFlag==0 && isfile(macFile)
            checkFlag = 1
        end
    end
    
    print("analysis_"*string(i)*" completed.")
    end

In [ ]:
# Main function
##----Parameters----############################################################
# Number of files is n_end - n_start
n_start = 1 # Start number
n_end = 250 # End number
# Ensure that counter is increasing!
if n_start > n_end
    n_start = 1
    n_end = 10
end

timeSteps = 100;
t = LinRange(0,1,timeSteps+1) # Time variable
outputPrecision = "5"

strainPath = "D:/Justin_File/3D_RVE/FFT/StrainData/"
strainNameFormat = "strainData_"
orientationPath = "D:/Justin_File/3D_RVE/FFT/OrientationData/"
orientationNameFormat = "orientationData_"
sizeRVENameFormat = "RVESize_"

dafPath = "D:/Justin_File/3D_RVE/FFT/DigimatFile/"
dafNameFormat = "analysis_"

finalTime = "1.0e+00"
maxTimeInc = string(1/timeSteps)
minTimeInc = string(1/timeSteps/10)
initialTimeInc = string(1/timeSteps)

youngFiber = "76e+09"
poissonFiber = "2.2e-01"
fiberVolumefraction = "1e-01" # With tensor orientation, this actual ref VF will be +0-50%
fiberDiameter = "1e-5"
fiberSize = "0.00024"
fiberOrientation = "tensor"

youngMatrix = "3.1e+09"
poissonMatrix = "3.5e-01"
yieldMatrix = "2.5e+07"
hardeningModulusMatrix = "2e+07"
hardeningExponentMatrix = "3.25e+02"
hardeningModulus2Matrix = "1.5e+08"
################################################################################

In [30]:
# IMPORTANT! Match significant digits of time vector to outputPrecision!
t = round.(t,sigdigits=parse(Int,outputPrecision))

# Generate strain path data. Optionally also generates orientation tensors.
for i = n_start:n_end
    noise = rand()
    n_drift = rand([1 2 5 10])
    # Calculate n_step by dividing timeStep by n_drift to ensure
    # constant length of time series
    n_step = div(timeSteps,n_drift) # div(x,y) to ensure int.
    EPS = 0.01 + rand()*0.04 # Maximum admissible component in 0.01 to 0.05
    if rand() > 0.9
        uni = true
    else
        uni = false
    end
    strainPathGenerator(noise,n_step,n_drift,EPS,t,strainPath*strainNameFormat*string(i),uni)
    randon3dOrientationTensorGenerator(fiberVolumefraction,
    orientationPath*orientationNameFormat*string(i),
    orientationPath*sizeRVENameFormat*string(i))
end

# Write *.daf files for every strain path.
for i = n_start:n_end
    local inPath = strainPath*strainNameFormat*string(i)*".txt"
    local outPath = dafPath*dafNameFormat*string(i)*".daf"
    local inPath2 = orientationPath*orientationNameFormat*string(i)*".txt"
    local inPath3 = orientationPath*sizeRVENameFormat*string(i)*".txt"
    writedaf(inPath,inPath2,inPath3,outPath,finalTime,maxTimeInc,minTimeInc,initialTimeInc,
    youngFiber,poissonFiber,fiberDiameter,fiberSize,fiberOrientation,
    youngMatrix,poissonMatrix,yieldMatrix,hardeningModulusMatrix,hardeningExponentMatrix,hardeningModulus2Matrix)
end

In [ ]:
# Run DIGIMAT simulation for every *.daf file.
for i = n_start:n_end
    maxTime = 70*60;
    checkTime = 5*60;
    RunDigimatFEwithTimeLimit(i, dafPath, maxTime, checkTime)
end